In [1]:
import xgboost as xgb
print(xgb.__version__)

2.0.3


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/anuvaagarwal/Documents/mlproject/Regression_ML_EndtoEnd/.venv/bin/python
2.0.3
/Users/anuvaagarwal/Documents/mlproject/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

/Users/anuvaagarwal/Documents/mlproject/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/Users/anuvaagarwal/Documents/mlproject/Regression_ML_EndtoEnd/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/Users/anuvaagarwal/Documents/mlproject/Regression_ML_EndtoEnd/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [6]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [8]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/Users/anuvaagarwal/Documents/mlproject/Regression_ML_EndtoEnd/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/11/23 13:23:58 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-11-23 13:23:58,964] A new study created in memory with name: no-name-7c516b75-92a8-4c6e-b2cf-fed997aa839d
[I 2025-11-23 13:24:03,892] Trial 0 finished with value: 73102.35373984821 and parameters: {'n_estimators': 269, 'max_depth': 7, 'learning_rate': 0.033798030009067524, 'subsample': 0.8848138043617988, 'colsample_bytree': 0.8034552007824753, 'min_child_weight': 4, 'gamma': 0.6372358692077695, 'reg_alpha': 0.09634100973371755, 'reg_lambda': 8.475366774204912e-08}. Best is trial 0 with value: 73102.35373984821.
[I 2025-11-23 13:24:08,074] Trial 1 finished with value: 75021.87593497743 and parameters: {'n_estimators': 345, 'max_depth': 5, 'learning_rate': 0.23556574509520067, 'subsample': 0.5944114533091552, 'colsample_bytree': 0.540275971074366, 'min_child_weight': 7, 'gamma': 4.998574877652635, 'reg_alpha': 0.022340242431340394, 'reg_lambda

Best params: {'n_estimators': 985, 'max_depth': 8, 'learning_rate': 0.04317920542826245, 'subsample': 0.8977305424835171, 'colsample_bytree': 0.5120304719631705, 'min_child_weight': 3, 'gamma': 2.070571816930866, 'reg_alpha': 7.6147500898130636e-06, 'reg_lambda': 8.031591009347776e-06}


In [9]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 30657.09982610076
RMSE: 69323.42466233562
R²: 0.9628618684974701


/Users/anuvaagarwal/Documents/mlproject/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:32:12] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
2025/11/23 13:32:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
